## Hyperparameter tuning

In [ ]:
import talos

def model_talos(train_X, train_Y, val_X, val_Y, params):
    vanilla_rnn = Sequential()
    
    for i in range(max(params['layers']-1, 0)):
        vanilla_rnn.add(LSTM(params['first_lstm'], return_sequences=True))

    vanilla_rnn.add(LSTM(params['first_lstm'],input_shape = (1, params['window_size']), activation='tanh', 
                         recurrent_activation='sigmoid', use_bias=True, kernel_initializer='glorot_uniform', 
                         recurrent_initializer='orthogonal', bias_initializer='zeros'))
    
    vanilla_rnn.add(Dense(1, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros'))
    
    vanilla_rnn.compile(params['optimizer'], loss=params['loss'])
    
    history = vanilla_rnn.fit(train_X, train_Y, epochs = params['epochs'], batch_size = params['batch_size'], 
                              verbose = 0,  validation_data=[val_X, val_Y])
    
    return history, vanilla_rnn



In [ ]:
p = {'optimizer': ['adam', 'sgd', 'rmsprop'],
     'loss': ['mse'],
     'batch_size': [32, 128, 1024],
     'epochs': [5, 10, 20],
     'layers' : [1, 2],
     'first_lstm' : [4, 8, 16, 32],
     'window_size' : [1, 24, 24*7]}

scan_object = talos.Scan(train_X, train_Y, model=model_talos, params=p, fraction_limit=0.2, val_split=0.15, experiment_name='1')


In [ ]:
# accessing the results data frame
scan_object.data.head()

# accessing epoch entropy values for each round
scan_object.learning_entropy

# access the summary details
scan_object.details

# use Scan object as input
analyze_object = talos.Analyze(scan_object)

# access the dataframe with the results
analyze_object.data

# four dimensional bar grid
analyze_object.plot_bars('layers', 'val_loss', 'window_size', 'optimizer')

# four dimensional bar grid
analyze_object.plot_bars('batch_size', 'val_loss', 'first_lstm', 'epochs')

# heatmap correlation
analyze_object.plot_corr('val_loss', [])